# Overall strategy:

#### 1) split into good and bad data. Currently only splitting based on > 50% fordelt. Need to split those who only deliver to 1 bedrift into bad as well. 
#### 2) for the bad data, use ml to guess the oms , distribute drkost based on profit level and oms. 
#### 3) merge this data back with the good data.
#### 4) run the regular AO with 99 on th entire df
#### 5) create an update file. Send to google cloud
#### 6) send to linux. Use the sas code to update isee

### How to organise functions in final notebook

#### Choose naring
#### Choose parameters for separating between good, bad dfs
#### Apply which machine learning model? Varehandels
#### Bring back everything to a final DF. Apply 999 logic to this df. 
#### Further updates to rad 0 level data (foretak) 
#### Need to think about what to do with reg_types for example reg_type 03

- (A) not 100% fordelt. automatically adjust. Create a list of things of foretak to check naring. 
- avanse
- 1000 feils
- IKT investering


- Need to bring in past years data and data from other sources. 

#### Create updatefile. 

## FEATURE ENGINEERING!



In [ ]:
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.cluster import DBSCAN
import pyarrow as pa
import pyarrow.parquet as pq
import gcsfs
import getpass
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import geopandas as gpd
import sgis as sg
import dapla as dp
import datetime
from dapla.auth import AuthClient
from dapla import FileClient
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import requests
from pyjstat import pyjstat


import sys

sys.path.append("../functions")
import kommune_pop
import kommune_inntekt

fs = FileClient.get_gcs_file_system()
import numpy as np


import warnings

warnings.filterwarnings("ignore")

# Årgang

In [ ]:
year = "2021"
fjor = "2020"

# get dynarev data

In [ ]:
fil_path = [
    f
    for f in fs.glob(
        f"gs://ssb-prod-noeku-data-produkt/eimerdb/nokubasen/skjemadata/aar={year}/skjema=RA-0174-1/*"
    )
    if f.endswith(".parquet")
]

# Use the ParquetDataset to read multiple files
dataset = pq.ParquetDataset(fil_path, filesystem=fs)
table = dataset.read()

# Convert to Pandas DataFrame
skjema = table.to_pandas()

In [ ]:
felt_id_values = [
    "V_ORGNR",
    "F_ADRESSE",
    "FJOR_NACE_B_T1",
    "TMP_SN2007_5",
    "B_KOMMUNENR",
    "REGTYPE",
    "B_SYSSELSETTING_SYSS",
    "TMP_NY_BDR_SYSS",
    "GJELDENDE_BDR_SYSS",
    "FJOR_SYSSEL_T1",
    "LONN_PST_AORDN",
    "GJELDENDE_LONN_KR",
    "LONN",
    "FJOR_LONN_KR_T1",
    "TMP_SNITTLONN",
    "FJOR_SNITTLONN_T1",
    "GJELDENDE_OMSETN_KR",
    "OMSETN_KR",
    "FJOR_OMSETN_KR_T1",
    "TMP_SNITTOMS",
    "FJOR_SNITTOMS_T1",
    "TMP_SALGSINT_BED",
    "TMP_FORBRUK_BED",
    "VAREKOST_BED",
    "GJELDENDE_DRIFTSK_KR",
    "DRIFTSKOST_KR",
    "FJOR_DRIFTSKOST_KR_T1",
    "NACEF_5",
    "SALGSINT",
    "FORBRUK",
    "TMP_NO_P4005",
    "TMP_AVPROS_ORGFORB",
    "ORGNR_N_1",
    "TMP_NO_OMSETN",
    "TMP_DRIFTSKOSTNAD_9010",
    "TMP_DRIFTSKOSTNAD_9910",
]

# Assuming `skjema` is your DataFrame and `felt_id_values` is your list of values
skjema = skjema[skjema["feltnavn"].isin(felt_id_values)]

In [ ]:
pivot_df = skjema.pivot_table(
    index=["id", "radnr", "lopenr"],
    columns="feltnavn",
    values="feltverdi",
    aggfunc="first",
)
pivot_df = pivot_df.reset_index()
pivot_df.columns = pivot_df.columns.str.lower()
pivot_df
# test = pivot_df['FJOR_OMSETN_KR_T1']
# test

In [ ]:
# Assuming your DataFrame is named pivot_df

# Create the 'foretak' DataFrame
foretak = pivot_df.loc[pivot_df["radnr"] == 0]

# Create the 'bedrift' DataFrame
bedrift = pivot_df.loc[pivot_df["radnr"] > 0]

In [ ]:
pd.set_option("display.max_columns", None)
# Assuming 'foretak' is your existing DataFrame

selected_columns = [
    "id",
    "lopenr",
    "forbruk",
    "nacef_5",
    "orgnr_n_1",
    "salgsint",
    "tmp_driftskostnad_9010",
    "tmp_driftskostnad_9910",
    "tmp_no_omsetn",
    "tmp_no_p4005",
]

foretak = foretak[selected_columns]

# Assuming 'foretak' is your DataFrame
foretak.rename(columns={"tmp_no_omsetn": "foretak_omsetning"}, inplace=True)


foretak = foretak.fillna(0)

foretak[["tmp_driftskostnad_9010", "tmp_driftskostnad_9910"]] = foretak[
    ["tmp_driftskostnad_9010", "tmp_driftskostnad_9910"]
].apply(pd.to_numeric, errors="coerce")

foretak["foretak_driftskostnad"] = foretak[
    ["tmp_driftskostnad_9010", "tmp_driftskostnad_9910"]
].max(axis=1)

# Drop the specified columns
foretak.drop(["tmp_driftskostnad_9010", "tmp_driftskostnad_9910"], axis=1, inplace=True)

In [ ]:
# Assuming 'bedrift' is your DataFrame
columns_to_drop = [
    "forbruk",
    "nacef_5",
    "orgnr_n_1",
    "salgsint",
    "tmp_driftskostnad_9010",
    "tmp_driftskostnad_9910",
    "tmp_no_omsetn",
    "tmp_no_p4005",
]

bedrift.drop(columns_to_drop, axis=1, inplace=True)

In [ ]:
# Assuming 'bedrift' is your DataFrame
columns_to_fill = ["omsetn_kr", "driftskost_kr"]

# Convert columns to numeric, replacing non-convertible values with NaN
bedrift[columns_to_fill] = bedrift[columns_to_fill].apply(
    pd.to_numeric, errors="coerce"
)

# Fill NaN values with 0 for the specified columns
bedrift[columns_to_fill] = bedrift[columns_to_fill].fillna(0)

# Group by 'id' and calculate the sum
grouped_bedrift = (
    bedrift.groupby("id")[["omsetn_kr", "driftskost_kr"]].sum().reset_index()
)

# Rename the columns
grouped_bedrift.rename(
    columns={"omsetn_kr": "tot_oms_fordelt", "driftskost_kr": "tot_driftskost_fordelt"},
    inplace=True,
)

# Merge the grouped DataFrame back to the original DataFrame based on 'id'
bedrift = pd.merge(bedrift, grouped_bedrift, on="id", how="left")

In [ ]:
# Assuming 'foretak' and 'bedrift' are your DataFrames
merged_df = pd.merge(foretak, bedrift, on=["id", "lopenr"], how="inner")
# merged_df.head()

In [ ]:
# Assuming 'merged_df' is your DataFrame

# Convert columns to numeric, replacing non-convertible values with NaN
merged_df["tot_oms_fordelt"] = pd.to_numeric(
    merged_df["tot_oms_fordelt"], errors="coerce"
)
merged_df["foretak_omsetning"] = pd.to_numeric(
    merged_df["foretak_omsetning"], errors="coerce"
)

# Calculate omsetning_percentage
merged_df["omsetning_percentage"] = (
    merged_df["tot_oms_fordelt"] / merged_df["foretak_omsetning"]
)

# Convert columns to numeric, replacing non-convertible values with NaN
merged_df["tot_driftskost_fordelt"] = pd.to_numeric(
    merged_df["tot_driftskost_fordelt"], errors="coerce"
)
merged_df["foretak_driftskostnad"] = pd.to_numeric(
    merged_df["foretak_driftskostnad"], errors="coerce"
)

# Calculate driftskostnader_percentage
merged_df["driftskostnader_percentage"] = (
    merged_df["tot_driftskost_fordelt"] / merged_df["foretak_driftskostnad"]
)

merged_df["driftskostnader_percentage"] = (
    merged_df["tot_driftskost_fordelt"] / merged_df["foretak_driftskostnad"]
).round(4)

# Fill NaN with a specific value (e.g., 0)
merged_df["driftskostnader_percentage"].fillna(0, inplace=True)
merged_df["omsetning_percentage"].fillna(0, inplace=True)

In [ ]:
# merged_df.shape

In [ ]:
# Assuming 'merged_df' is your DataFrame

# Create the 'Good' DataFrame
good_temp_df = merged_df[
    (merged_df["omsetning_percentage"] >= 0.5)
    & (merged_df["driftskostnader_percentage"] >= 0.5)
]

# Create 'bedrift_count' and 'distribution_count'
good_temp_df["bedrift_count"] = good_temp_df.groupby("orgnr_n_1")[
    "orgnr_n_1"
].transform("count")
good_temp_df["distribution_count"] = good_temp_df.groupby("orgnr_n_1")[
    "omsetn_kr"
].transform(lambda x: (x > 0).sum())

# Create 'bad_temp' DataFrame based on conditions
bad_temp = good_temp_df[
    (good_temp_df["bedrift_count"] > 5) & (good_temp_df["distribution_count"] <= 2)
]

# Create 'good_df' by excluding rows from 'bad_temp'
good_df = (
    pd.merge(good_temp_df, bad_temp, how="outer", indicator=True)
    .query('_merge == "left_only"')
    .drop("_merge", axis=1)
)


# Create the 'Mixed' DataFrame
onlygoodoms = merged_df[
    (
        (merged_df["omsetning_percentage"] > 0.5)
        & (merged_df["driftskostnader_percentage"] <= 0.5)
    )
]

onlygooddriftskostnader = merged_df[
    (
        (merged_df["driftskostnader_percentage"] > 0.5)
        & (merged_df["omsetning_percentage"] <= 0.5)
    )
]

# Create the 'Bad' DataFrame
bad_df = merged_df[
    (merged_df["omsetning_percentage"] <= 0.5)
    & (merged_df["driftskostnader_percentage"] <= 0.5)
]
bad_df = pd.concat([bad_df, bad_temp]).drop_duplicates(keep=False)
bad_df = pd.concat([bad_df, onlygooddriftskostnader]).drop_duplicates(keep=False)

In [ ]:
merged_df.shape

In [ ]:
good_df.shape

In [ ]:
onlygoodoms.shape

In [ ]:
onlygooddriftskostnader.shape

In [ ]:
bad_df.shape

good distribution delivered (both oms and dkost): 83.33%
onlygoodoms: 11.96%
onlygooddriftskostnader: 0.693%
bad (bad distribution of both oms and dkost) : 4.02% 

# Step 1. 

Fill all bad df with NAN. Fill only onlygooddriftskostnader with NAN for oms. For onlygoodoms use profit margin to impute driftskostnader.

In [ ]:
# Fill bad_df with NaN
bad_df["omsetn_kr"] = np.nan
bad_df["driftskost_kr"] = np.nan

bad_df["new_oms"] = bad_df["omsetn_kr"]

# Fill onlygooddriftskostnader 'oms' with NAN.
onlygooddriftskostnader["omsetn_kr"] = np.nan

onlygooddriftskostnader["new_oms"] = onlygooddriftskostnader["omsetn_kr"]

good_df["new_oms"] = good_df["omsetn_kr"]

# Correct driftskostnader for onlygoodoms

onlygoodoms["oms_share"] = onlygoodoms["omsetn_kr"] / onlygoodoms[
    "tot_oms_fordelt"
].round(5)

# Round the values to whole numbers before assigning to the new columns
onlygoodoms["new_oms"] = (
    (onlygoodoms["oms_share"] * onlygoodoms["foretak_omsetning"]).round(0).astype(int)
)

onlygoodoms["margin"] = (
    onlygoodoms["foretak_driftskostnad"] / onlygoodoms["foretak_omsetning"]
)

onlygoodoms["old_drkost"] = onlygoodoms["driftskost_kr"]

onlygoodoms["new_drkost"] = onlygoodoms["margin"] * onlygoodoms["new_oms"]

onlygoodoms["driftskost_kr"] = onlygoodoms["new_drkost"]

In [ ]:
bad_df

 

# Step 2: 

merge bad dfs back into good_df

In [ ]:
# Assuming 'onlygoodoms','bad_df', 'good_df', and 'onlygooddriftskostnader' are your DataFrames
# Merge all DataFrames into 'imputer' in a single step
merged_df = pd.concat(
    [onlygoodoms, bad_df, good_df, onlygooddriftskostnader], ignore_index=True
)

# Assuming 'imputer' is your DataFrame and you want to select specific columns
selected_columns = [
    "id",
    "lopenr",
    "forbruk",
    "nacef_5",
    "orgnr_n_1",
    "salgsint",
    "foretak_omsetning",
    "tmp_no_p4005",
    "foretak_driftskostnad",
    "radnr",
    "gjeldende_bdr_syss",
    "fjor_driftskost_kr_t1",
    "fjor_lonn_kr_t1",
    "fjor_syssel_t1",
    "gjeldende_lonn_kr",
    "new_oms",
    "b_kommunenr",
]

# Create a new DataFrame with only the selected columns
imputer = merged_df[selected_columns].copy()

merged_df.head(100)

# Assuming 'df' is your DataFrame
# nan_count = imputer['new_oms'].isna().sum()
# print("Number of NaN values in 'new_oms':", nan_count)

# step 3 . 

Feature engineering:

- delta foretak oms
- delta kommune population
- delta kommune income
- change in employees
- consumer confidence index
- inflation
- gdp growth
- past data
- past years sales * foretak increase
- past years sales * kommune pop
- past years sales * kommune income

## Kommune belfolkning delta

In [ ]:
kommune_pop_aar = kommune_pop.fetch_population_data(year)
kommune_pop_fjor = kommune_pop.fetch_population_data(fjor)
kommune_befolk = kommune_pop.befolkning_behandling(kommune_pop_aar, kommune_pop_fjor)

kommune_befolk.head()

## kommune inntekt delta

In [ ]:
kommune_innt_aar = kommune_inntekt.fetch_inntekt_data(year)
kommune_innt_fjor = kommune_inntekt.fetch_inntekt_data(fjor)
kommune_inntekt = kommune_inntekt.inntekt_behandling(
    kommune_innt_aar, kommune_innt_fjor
)
kommune_inntekt.head()

# Step 4
Use IterativeImputer to imput Nan values.

In [ ]:
# Assuming 'imputer' is your DataFrame and 'imputer_oms' is the variable you want to impute
# Extract the features for imputation (excluding 'imputer_oms')
features = [col for col in imputer.columns if col != "imputer_oms"]

# Initialize the IterativeImputer
imputer_model = IterativeImputer(random_state=0)

# Fit the imputer model on the features with missing values
imputer_model.fit(imputer[features])

# Impute missing values in the 'imputer_oms' column
imputer["imputer_oms"] = imputer_model.transform(imputer[features])[
    :, features.index("imputer_oms")
]

# Step 5

Use tools to evaluate the model. 

# Step 6

Run it all through the regular AO system. 

# Links to different treatment for each type of df. All will be merged into one dataframe at the end. 
### [1. Only Good Oms DF](#onlygoodoms) | Can basically just use the profit margin of the foretak to work out driftskostnader |
### [2. Bad DF](#baddf) | Need to incorporate the machine learning here |
### [3. Merge into good DF](#merge) | Basically a run through of 99. Eventual plan is to have the other dfs mergedinto good_df |
### [4. Good DF](#gooddf) | Basically a run through of 99. Eventual plan is to have the other dfs mergedinto good_df |


<a id='onlygoodoms'></a>
# 1. Only Good Oms DF

#### Uses the profit margin

In [ ]:
onlygoodoms["oms_share"] = onlygoodoms["omsetn_kr"] / onlygoodoms[
    "tot_oms_fordelt"
].round(5)

# Round the values to whole numbers before assigning to the new columns
onlygoodoms["new_oms"] = (
    (onlygoodoms["oms_share"] * onlygoodoms["foretak_omsetning"]).round(0).astype(int)
)

onlygoodoms["margin"] = (
    onlygoodoms["foretak_driftskostnad"] / onlygoodoms["foretak_omsetning"]
)

onlygoodoms["old_drkost"] = onlygoodoms["driftskost_kr"]

onlygoodoms["new_drkost"] = onlygoodoms["margin"] * onlygoodoms["new_oms"]

onlygoodoms["driftskost_kr"] = onlygoodoms["new_drkost"]

onlygoodoms.head()

<a id='gooddf'></a>
# 4. Good DF AO

### rette oms

In [ ]:
# pd.set_option("display.max_rows", None)


good_df["oms_share"] = good_df["omsetn_kr"] / good_df["tot_oms_fordelt"].round(5)

# Round the values to whole numbers before assigning to the new columns
good_df["new_oms"] = (
    (good_df["oms_share"] * good_df["foretak_omsetning"]).round(0).astype(int)
)


# good_df.head()

### rette salgsint og forbruk

data salgsint_forbruk;
	set enhetene_brukes6  
		(
		where=(rad_nr > 0)
		keep=
		orgnr_foretak 
		lopenr
		rad_nr
		tmp_ny_omsetn
		salgsint
		forbruk
		naring
		foretaksnaring
			);
	vhbed=0;

	if substr(naring,1,2) in &w_naring_vh. then
		vhbed=1;

	if naring in &w_nace1_ikke_vh. then
		vhbed=0;

	if substr(naring,1,4) in &w_nace2_ikke_vh. then
		vhbed=0;
run;

w_naring_vh = ('45', '46', '47')
w_nace1_ikke_vh = 45.403
w_nace2_ikke_vh =  ('45.2', '46.1')


In [ ]:
# 1
# Define the values for w_naring_vh, w_nace1_ikke_vh, and w_nace2_ikke_vh
w_naring_vh = ("45", "46", "47")
w_nace1_ikke_vh = "45.403"
w_nace2_ikke_vh = ("45.2", "46.1")

enhetene_brukes = good_df.copy()

# Filter the DataFrame based on conditions and create vhbed variable
enhetene_brukes["vhbed"] = 0

# Check if the first two characters of 'naring' are in w_naring_vh
enhetene_brukes.loc[
    enhetene_brukes["tmp_sn2007_5"].str[:2].isin(w_naring_vh), "vhbed"
] = 1

# Check if 'naring' is in w_nace1_ikke_vh
enhetene_brukes.loc[enhetene_brukes["tmp_sn2007_5"] == w_nace1_ikke_vh, "vhbed"] = 0

# Check if the first four characters of 'naring' are in w_nace2_ikke_vh
enhetene_brukes.loc[
    enhetene_brukes["tmp_sn2007_5"].str[:4].isin(w_nace2_ikke_vh), "vhbed"
] = 0

# enhetene_brukes.head()

In [ ]:
# 2
salgsint_forbruk = enhetene_brukes[
    [
        "orgnr_n_1",
        "lopenr",
        "v_orgnr",
        "forbruk",
        "salgsint",
        "radnr",
        "nacef_5",
        "tmp_sn2007_5",
        "new_oms",
        "vhbed",
    ]
]

# salgsint_forbruk.head()

In [ ]:
# 3

har = salgsint_forbruk[salgsint_forbruk.groupby("orgnr_n_1")["vhbed"].transform("any")]
# Extract the 'orgnr_n_1' column
har = har[["orgnr_n_1"]]

# Remove duplicates
har.drop_duplicates(inplace=True)

# har.head()

In [ ]:
# 4

ikke_har = salgsint_forbruk[
    ~salgsint_forbruk.groupby("orgnr_n_1")["vhbed"].transform("any")
]
ikke_har = ikke_har[["orgnr_n_1"]]
ikke_har.drop_duplicates(inplace=True)

ikke_har["ikkevbed"] = 1

# ikke_har.head()

In [ ]:
# 5


# Merge ikke_har into salgsint_forbruk with a left join on the 'id' column
salgsint_forbruk_update1 = pd.merge(
    salgsint_forbruk, ikke_har, on="orgnr_n_1", how="left"
)

# salgsint_forbruk_update1['ikkevbed'].fillna(0, inplace=True)

# Update 'vhbed' to 1 where 'ikkevbed' is 1
salgsint_forbruk_update1.loc[salgsint_forbruk_update1["ikkevbed"] == 1, "vhbed"] = 1

# salgsint_forbruk_update1.head()

In [ ]:
# 6
# Assuming your original DataFrame is named salgsint_forbruk_update1
# Replace 'new_oms', 'orgnr_foretak', 'lopnr', 'vhbed' with the actual column names in your DataFrame

# Create sum1 DataFrame for vhbed=1
sum1 = (
    salgsint_forbruk_update1[salgsint_forbruk_update1["vhbed"] == 1]
    .groupby(["orgnr_n_1", "lopenr"])["new_oms"]
    .sum()
    .reset_index()
)
sum1.rename(columns={"new_oms": "sumoms_vh"}, inplace=True)

# Create sum2 DataFrame for vhbed=0
sum2 = (
    salgsint_forbruk_update1[salgsint_forbruk_update1["vhbed"] == 0]
    .groupby(["orgnr_n_1", "lopenr"])["new_oms"]
    .sum()
    .reset_index()
)
sum2.rename(columns={"new_oms": "sumoms_andre"}, inplace=True)

In [ ]:
# Perform an outer join of sum1 and sum2 by 'orgnr_foretak' and 'lopnr'
sum3 = pd.merge(sum1, sum2, on=["orgnr_n_1", "lopenr"], how="outer")
# sum3.head()

In [ ]:
# 8

salgsint_forbruk_update2 = pd.merge(
    salgsint_forbruk_update1, sum3, on=["orgnr_n_1", "lopenr"], how="outer"
)
# salgsint_forbruk_update2.head()

In [ ]:
# 9 & 10
# Assuming 'orgnr_n_1', 'lopenr', and 'rad_nr' are the actual column names
# Replace them with the actual names in your DataFrame

# Sort the DataFrame by 'orgnr_n_1', 'lopenr', and 'rad_nr'
salgsint_forbruk_update2.sort_values(by=["orgnr_n_1", "lopenr", "radnr"], inplace=True)

salgsint_forbruk_update2.sort_values(by=["orgnr_n_1", "lopenr", "vhbed"], inplace=True)

# Display the sorted DataFrame
# salgsint_forbruk_update2.head()

In [ ]:
# 11

# Assuming your DataFrame is named salgsint_forbruk
# Replace 'orgnr_foretak', 'lopenr', 'vhbed', 'vhf' with the actual column names

# Sort the DataFrame by 'orgnr_foretak' and 'lopenr'

salgsint_forbruk_update3 = salgsint_forbruk_update2.copy()

salgsint_forbruk_update3.sort_values(by=["orgnr_n_1", "lopenr"], inplace=True)

# Create a new variable 'vhf' based on the values of 'vhbed'
salgsint_forbruk_update3["vhf"] = salgsint_forbruk_update3.groupby(
    ["orgnr_n_1", "lopenr"]
)["vhbed"].transform("first")

# Retain the value of 'vhf' from the first observation in each group
salgsint_forbruk_update3["vhf"] = salgsint_forbruk_update3.groupby(
    ["orgnr_n_1", "lopenr"]
)["vhf"].transform("first")

# Apply labels to the variables
salgsint_forbruk_update3["vhbed"] = salgsint_forbruk_update3["vhbed"].astype(str)
salgsint_forbruk_update3["vhf"] = salgsint_forbruk_update3["vhf"].astype(str)

label_map_vhbed = {"1": "varehandelsbedrift", "0": "annen type bedrift"}
label_map_vhf = {
    "1": "foretaket har kun varehandelsbedrifter eller ingen",
    "0": "har varehandel og annen bedrift (blandingsnæringer)",
}

salgsint_forbruk_update3["vhbed"] = salgsint_forbruk_update3["vhbed"].map(
    label_map_vhbed
)
salgsint_forbruk_update3["vhf"] = salgsint_forbruk_update3["vhf"].map(label_map_vhf)

# salgsint_forbruk_update3.head()

In [ ]:
# 11

# Assuming your DataFrame is named salgsint_forbruk
# Replace 'vhf' with the actual name of your column

# Filter rows where vhf is 'foretaket har kun varehandelsbedrifter eller ingen'
vhf_condition = (
    salgsint_forbruk_update3["vhf"]
    == "foretaket har kun varehandelsbedrifter eller ingen"
)
vhf_df = salgsint_forbruk_update3.loc[vhf_condition]

# Filter rows where vhf is not 'foretaket har kun varehandelsbedrifter eller ingen'
andre_df = salgsint_forbruk_update3.loc[~vhf_condition]

# vhf_df.head()
# andre_df.head()

In [ ]:
# 12

vhf_df["nokkel"] = vhf_df["new_oms"] / vhf_df["sumoms_vh"]

# Convert 'salgsint' column to numeric
vhf_df["salgsint"] = pd.to_numeric(vhf_df["salgsint"], errors="coerce")
vhf_df["forbruk"] = pd.to_numeric(vhf_df["forbruk"], errors="coerce")


vhf_df["bedr_salgsint"] = round(vhf_df["salgsint"] * vhf_df["nokkel"])
vhf_df["bedr_forbruk"] = round(vhf_df["forbruk"] * vhf_df["nokkel"])

# vhf_df.head()

In [ ]:
# 13

andre_df["forbruk"] = pd.to_numeric(andre_df["forbruk"], errors="coerce")
andre_df["salgsint"] = pd.to_numeric(andre_df["salgsint"], errors="coerce")


# Assuming 'andre' is your DataFrame
andre_df["avanse"] = andre_df["forbruk"] / andre_df["salgsint"]

# Filter rows where vhbed is 1
vh_bedriftene = andre_df[andre_df["vhbed"] == "varehandelsbedrift"].copy()

# Calculate 'nokkel', 'bedr_salgsint', and 'bedr_forbruk' for vh-bedriftene
vh_bedriftene["nokkel"] = vh_bedriftene["new_oms"] / vh_bedriftene["sumoms_vh"]
vh_bedriftene["bedr_salgsint"] = round(
    vh_bedriftene["salgsint"] * vh_bedriftene["nokkel"]
)
vh_bedriftene.loc[
    vh_bedriftene["bedr_salgsint"] > vh_bedriftene["new_oms"], "bedr_salgsint"
] = vh_bedriftene["new_oms"]
vh_bedriftene["bedr_forbruk"] = round(
    vh_bedriftene["bedr_salgsint"] * vh_bedriftene["avanse"]
)

# Summarize vh-bedriftene
brukt1 = (
    vh_bedriftene.groupby(["orgnr_n_1", "lopenr"])
    .agg({"bedr_salgsint": "sum", "bedr_forbruk": "sum"})
    .reset_index()
)

# Merge summarized values back to 'andre'
andre = pd.merge(andre_df, brukt1, on=["orgnr_n_1", "lopenr"], how="left")

# Calculate 'resten1' and 'resten2'
andre["resten1"] = andre["salgsint"] - andre["bedr_salgsint"]
andre["resten2"] = andre["forbruk"] - andre["bedr_forbruk"]

# Filter rows where vhbed is not 1
blanding_av_vh_og_andre = andre[andre["vhbed"] != "varehandelsbedrift"].copy()

# Calculate 'nokkel', 'bedr_salgsint', and 'bedr_forbruk' for blending of vh and other industries
blanding_av_vh_og_andre["nokkel"] = (
    blanding_av_vh_og_andre["new_oms"] / blanding_av_vh_og_andre["sumoms_andre"]
)
blanding_av_vh_og_andre["bedr_salgsint"] = round(
    blanding_av_vh_og_andre["resten1"] * blanding_av_vh_og_andre["nokkel"]
)
blanding_av_vh_og_andre["bedr_forbruk"] = round(
    blanding_av_vh_og_andre["resten2"] * blanding_av_vh_og_andre["nokkel"]
)

# Combine the two subsets back into 'andre'
andre = pd.concat([vh_bedriftene, blanding_av_vh_og_andre], ignore_index=True)

andre.sort_values(by=["orgnr_n_1", "lopenr"], inplace=True)

oppdatere_hv = pd.concat([vhf_df, andre], ignore_index=True)

oppdatere_hv = oppdatere_hv[
    ["orgnr_n_1", "lopenr", "radnr", "bedr_forbruk", "bedr_salgsint"]
]

# oppdatere_hv.head()

In [ ]:
enhetene_brukes2 = pd.merge(
    enhetene_brukes, oppdatere_hv, on=["orgnr_n_1", "lopenr", "radnr"]
)

# enhetene_brukes2.head()

In [ ]:
rettes = enhetene_brukes2.copy()

rettes["oms"] = rettes["new_oms"]
rettes["driftsk"] = rettes["gjeldende_driftsk_kr"]

# Convert columns to numeric
rettes["tot_driftskost_fordelt"] = pd.to_numeric(
    rettes["tot_driftskost_fordelt"], errors="coerce"
)
rettes["driftsk"] = pd.to_numeric(rettes["driftsk"], errors="coerce")


rettes["drkost_share"] = rettes["driftsk"] / rettes["tot_driftskost_fordelt"]

rettes["new_drkost"] = rettes["drkost_share"] * rettes["foretak_driftskostnad"]

# rettes.head()

# Kontrol Drkost for forbuk og lønn. 

In [ ]:
rettes2 = rettes.copy()
rettes2["drkost_temp"] = rettes2["new_drkost"]

# Fill NaN in 'drkost_temp' with 0
rettes2["drkost_temp"] = rettes2["drkost_temp"].fillna(0)

rettes2["gjeldende_lonn_kr"] = pd.to_numeric(
    rettes2["gjeldende_lonn_kr"], errors="coerce"
).fillna(0)
rettes2["bedr_forbruk"] = pd.to_numeric(
    rettes2["bedr_forbruk"], errors="coerce"
).fillna(0)


rettes2["lonn_+_forbruk"] = rettes2["gjeldende_lonn_kr"] + rettes2["bedr_forbruk"]

# Perform the if operation
condition = rettes2["drkost_temp"] < rettes2["lonn_+_forbruk"]
rettes2["drkost_temp"] = np.where(
    condition, rettes2["lonn_+_forbruk"], rettes2["drkost_temp"]
)
rettes2["theif"] = np.where(condition, 1, 0)

# rettes2.head()

### Create df with a list of orgnr_foretak that have thieves. 

In [ ]:
# Filter the DataFrame
dkvars = rettes2[rettes2.groupby("orgnr_n_1")["theif"].transform("any")]

# dkvars.shape

In [ ]:
# Calculate 'utskudd'
dkvars["utskudd"] = (
    dkvars["new_drkost"] - dkvars["gjeldende_lonn_kr"] - dkvars["bedr_forbruk"]
)
dkvars["utskudd"] = abs(dkvars["utskudd"])

# Keep selected columns
columns_to_keep = [
    "orgnr_n_1",
    "lopenr",
    "radnr",
    "utskudd",
    "new_drkost",
    "drkost_temp",
    "theif",
    "gjeldende_lonn_kr",
    "bedr_forbruk",
]
dkvars = dkvars[columns_to_keep]

# Display the resulting DataFrame
# dkvars.head()

In [ ]:
# Calculate sum of 'utskudd' grouped by 'orgnr_foretak', 'lopenr', and 'tyv'
sum7b = dkvars.groupby(["orgnr_n_1", "lopenr", "theif"])["utskudd"].sum().reset_index()

# Display the result
# sum7b.head()

In [ ]:
# Transpose the result
sum7b_transposed = sum7b.pivot(
    index=["orgnr_n_1", "lopenr"], columns="theif", values="utskudd"
).reset_index()

# Rename columns as per SAS code
sum7b_transposed.rename(columns={0: "thief0", 1: "thief1"}, inplace=True)

sum7b_transposed = sum7b_transposed[["orgnr_n_1", "lopenr", "thief0", "thief1"]]

# Display the transposed result
# sum7b_transposed.head()

In [ ]:
# merge sums
dkvars_2 = pd.merge(dkvars, sum7b_transposed, on=["orgnr_n_1", "lopenr"], how="inner")
# dkvars_2.head()

In [ ]:
# Apply conditional logic
pd.set_option("display.float_format", "{:.2f}".format)
dkvars_2["andel1"] = np.where(
    dkvars_2["theif"] == 0, dkvars_2["utskudd"] / dkvars_2["thief0"], np.nan
)
dkvars_2["andel2"] = np.where(
    dkvars_2["theif"] == 0, np.round(dkvars_2["andel1"] * dkvars_2["thief1"]), np.nan
)
# dkvars_2['new_drkost'] = np.where(dkvars_2['theif'] == 0, np.sum(dkvars_2['drkost_temp'] - dkvars_2['andel2'], axis=0), dkvars_2['drkost_temp'])
dkvars_2["new_drkost"] = np.where(
    dkvars_2["theif"] == 0,
    dkvars_2["drkost_temp"] - dkvars_2["andel2"],
    dkvars_2["drkost_temp"],
)

# Keep selected columns
columns_to_keep = ["orgnr_n_1", "lopenr", "radnr", "new_drkost"]
dkvars_3 = dkvars_2[columns_to_keep]

# dkvars_2.head(50)

good_final = dkvars_2.copy()

Steps:

1. Define the varehandel næringer that need salgsint and forbruk. 
2. Create a df called salgsint_forbruk that has the forbruk of the foretak, the salgsint of the foretak, orgnr_foretak, rad_nr, naring, foretaksnaring, the new oms and a column called vhbed which indicates 1 if it is a varehandel bedrift and 0 if it is not. 
3. create another df called har that is simply a list of orgnr_foretak where there is at least one bedrift that is in varehandel. Remove duplicates. 
4. create another df called ikke_har that is a list of orgnr_foretak that has 0 bedrifter in varehandel. Create a new variable called ikkevbed = 1. Remove duplicates. 
5. Update salgint_forbruk by merging salgsint_forbruk and ikke with a left join. If ikkevhbed=1 then vhbed=1. Basically what is happenign is that if a foretak has no varehandel, then salgsint will be distributed without bias to varehandel bedrifter. 
6. Create two new dfs. One is called sum1. This sums up all new_oms grouped by orgnr_foretak and lopnr for only if vbed=1. This new variable will be called sumoms_vh. The second df will be called sum2 and will be for only where vhbed=0. This new sum will be a variable called sumoms_andre. 
7. create a new df called sum3 that merges sum1 and sum2 by orgnr_foretak and lopnr. Outerjoin. 
8. update salgsint_forbruk by performing an outerjoin with sum3 by orgnr_foretak and lopnr. Outerjoin. 
9. Sort by orgnr_foretak, lopenr and rad_nr.
10. VERY IMPORTANt. SORT BY ORGNR_FORETAK , LOPENR and VHBED. 
11. data salgsint_forbruk;
	set salgsint_forbruk;
	by orgnr_foretak lopenr;

	if first.orgnr_foretak and first.lopenr then
		vhf=vhbed;
	retain vhf;
	label
		vhbed ='1=varehandelsbedrift, 0 er annen type bedrift'
		vhf   ='1=foretaket har kun varehandelsbedrifter eller ingen , 0=har varehandel og annen bedrift (blandingsnæringer)'
	;
run;

proc sort data=salgsint_forbruk;
	by orgnr_foretak lopenr rad_nr;
run;

*(6) deler opp i rene varehandelsforetak og i andre fioretak;
data vhf  andre;
	set salgsint_forbruk;

	if vhf=1 then
		output vhf;
	else output andre;
run;

Basically what is happening here, is that we are creating two tables. One that has foretak that ONLY have varehandel bedrift OR foretak that have 0 varehandle bedrifter. 

The two tables will be called vhf and andre. 

12. for vhf distribute salgsint and forbruk :

data vhf;
	set vhf;
	nokkel = tmp_ny_omsetn / sumoms_vh;
	bedr_salgsint = round(salgsint * nokkel);
	bedr_forbruk  = round(forbruk * nokkel);
run;

13. for andre. 

data andre;
	set andre;
	avanse=forbruk / salgsint;

	*vh-bedriftene;
	if vhbed=1 then
		do;
			nokkel = tmp_ny_omsetn / sumoms_vh;
			bedr_salgsint = round(salgsint * nokkel);

			if bedr_salgsint > tmp_ny_omsetn then
				bedr_salgsint = tmp_ny_omsetn;
			bedr_forbruk  = round(bedr_salgsint * avanse);
		end;
run;

*(8b) summerere vh-bedriftene;
proc sql stimer;
	create table brukt1 
		as select orgnr_foretak, lopenr, sum(bedr_salgsint) as brukt1, sum(bedr_forbruk) as brukt2
			from andre 
				where vhbed=1
					group by orgnr_foretak, lopenr
						order by orgnr_foretak, lopenr
	;
	create table andre  
		as select a.*, b.*
			from andre  a, brukt1  b
				where a.orgnr_foretak = b.orgnr_foretak and a.lopenr = b.lopenr
					order by a.orgnr_foretak, a.lopenr, a.rad_nr
	;
quit;

*(8c) sluttfordeler for de foretakene som har blanding av vh og andre næringer;
data andre;
	set andre;
	resten1 = sum(salgsint, -brukt1);
	resten2 = sum(forbruk,-brukt2);

	if vhbed ne 1 then
		do;
			nokkel = tmp_ny_omsetn / sumoms_andre;
			bedr_salgsint = round(resten1 * nokkel);
			bedr_forbruk  = round(resten2 * nokkel);
		end;
run;

# What is left to do:

### Correct salgsint and forbruk for good_df. (instructions in cell above) Look at the program in sas as well to know how to do it. 

### Correct drkost for good_df. Have to make sure it passes with salary and forbruk 

### Do the same for the mixed_dfs. (good oms, bad costs etc etc ) Will need to selectivly treat based on what is good and what isnt. Maybe its worth just fixing bad oms. Bad costs can be fixed based on oms

### Implement a the ml models for the bad_df. Maybe make it so you can choose which one to use. 